# 1. Business Understanding

# 2. Exploratory Data Analysis

# 3. Baseline Model

# 4. Iteration 1: Feature Engineering

# 5. Iteration 2: Model Improvement

# 6. Final Model Selection

# 7. Test Predictions